In [ ]:
using JuMP               # To write the optimisation models
using Cbc, Clp           # Solvers 
using Test               # Testing package
using JLD2;              # File I/O

# Homework 4

## Problem description

We need to fulfil the demand of clients using different servers. The demand and set of clients are unknown when making the decision on which servers to use. Consider the parameters:
- $C_j$ - cost of installing server $j$
- $P_s$ - probability of scenario $s$
- $F$ - cost of unmet demand (same for all clients $i$, servers $j$ and scenarios $s$)
- $Q_{ij}$ - benefit per one unit of demand of client $i$ served by server $j$
- $V$ - maximum allowed number of servers
- $D_{is}$ - demand of client $i$ in scenario $s$
- $U$ - maximum capacity of a server (same for all servers and scenarios)
- $H_{is}$ - a binary variable with value 1 if client $i$ is active in scenario $s$, i.e., the demand $D_{is}$ has to be fulfilled

Let the variables be

- $x_j$ - binary variable with value 1 if server $j$ is made available, i.e., built or installed
- $y_{ijs}$ - the proportion of demand $D_{is}$ fulfilled by server $j$. The total demand of $i$ served by $j$ is thus $y_{ijs} \times D_{is}$
- $z_{js}$ - capacity shortage for server $j$. If demand is not met otherwise, any server $j$ can procure emergency capacity at a price $F$.


The model is then given by:

\begin{align}
    \min_{x_j, z_{js}, y_{ijs}} & \sum_{j \in J} C_j x_j + \sum_{s} P_s \left( - \sum_{i \in I,j \in J}Q_{ij}D_{is}y_{ijs} + \sum_{j \in J} Fz_{js} \right) \\
    \text{s.t.: } & \sum_{j \in J} x_j \leq V   & (t)\\
    & \sum_{i \in I} D_{is}y_{ijs} - z_{js} \leq Ux_j, \forall j \in J, s \in S  & (u_{js})\\
    & \sum_{j \in J} y_{ijs} = H_{is}, \forall i \in I, s \in S  & (v_{is}) \\
    & x_j \in \{0,1\}, \ \forall j \in J \\
    & y_{ijs} \geq 0, \ \forall i \in I, \forall j \in J, \forall s \in S \\
    & z_{js} \geq 0, \ \forall j \in J, \forall s \in S
\end{align}

, where $t$, $u_{js}$, and $v_{is}$ are the dual variables related to the constraints in the model.

In [ ]:
struct Instance
    # sets
    I  # Set of clients
    J  # Set of servers
    S  # Set of scenarios
    # Parameters 
    V  # Max number of servers
    P  # Scenario probabilities
    H  # 1 if client requires service
    C  # Cost of locating server at j
    F  # Cost of unmet demand
    D  # Demand in location i served from server j
    Q  # Benefit per unit of demand served
    U  # Maximum server capacity (same for all servers)
    loc_i # Coordinates of clients
    loc_j # Coordinates of servers
end

In [ ]:
f = jldopen("hw4_ins.jld2")
ins = nothing
try
    ins = f["ins"]
finally
    close(f)
end;

# Task 1: implementing the large-scale model

## Model construction 

In the following, the full model must be implemented and solved using Cbc.

In [ ]:
function generate_full_problem(ins)
    I = ins.I 
    J = ins.J
    S = ins.S
    V = ins.V 
    P = ins.P
    H = ins.H
    C = ins.C
    F = ins.F
    D = ins.D
    Q = ins.Q
    U = ins.U
    # Write the full model in JuMP

    # Initialize model
    m = Model(Cbc.Optimizer)
    
    # TODO: add your code here
    
    # Return the generated model
    return m
end

In [ ]:
fullmodel = generate_full_problem(ins)
# set_silent(fullmodel)
optimize!(fullmodel)

In [ ]:
# Examine the solutions
@show x_bar = Int.(round.(value.(fullmodel[:x]).data))
@show opt_z = sum(value.(fullmodel[:z]))
@show objective_value(fullmodel);

# Task 2: Benders decomposition

## Benders main

Formulate the initial main problem for the decomposition. Use a single variable $\theta$ for representing the subproblem value.

In [ ]:
## Benders decomposition

## Generates the main problem
function generate_main(ins)
    
    J = ins.J
    V = ins.V
    C = ins.C
     
    main = Model(Cbc.Optimizer)
    set_silent(main)
    
    # TODO: add your code here
    return main  
end

In [ ]:
# Solve the main problem
function solve_main(ins, main)
    optimize!(main)
    return value.(main[:x]), value(main[:θ]), objective_value(main)    
end

## Subproblem

Formulate the primal subproblem with corresponding objective value represented by the variable $\theta$ in the main problem. The primal subproblem is not used in the decomposition algorithm, but you will use it to test your implementation of the dual subproblem. It might also be easier to start by formulating the primal problem and then work from there to obtain the its dual formulation.

In [ ]:
# Generate and solve the primal subproblem for a given x_bar. For test purposes only; if the dual is correct, the objective value of
# the dual subproblem must be the same as this.
function generate_and_solve_primal_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem    
    sub = Model(Clp.Optimizer)
    set_silent(sub)
    
    # TODO: add your code here
    optimize!(sub)
    return objective_value(sub)
    
end

## Dual subproblem

Formulate the dual subproblem. Consider the dual variables indicated in the fullmodel as we are expecting you to use the same names. Hint: You can find the conversion rules for primal and dual problems in Lecture 5.

In [ ]:
## Define Benders subproblem
function generate_and_solve_dual_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem
    sub_dual = Model(Clp.Optimizer)
    set_silent(sub_dual)
    
    # TODO: add your code here
    optimize!(sub_dual)
    
    u_bar = value.(sub_dual[:u])                     
    v_bar = value.(sub_dual[:v])                     
    opt_value = objective_value(sub_dual)
    
    return u_bar, v_bar, opt_value
end

## Benders cut

Formulate the Benders optimality cut. Remember to explain in your report why you only need to consider one type of cut.

In [ ]:
# Add the Benders cut, given current dual values
function add_benders_cut(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S
    
    x = main[:x]
    θ = main[:θ]
    
    @constraint(main, 
    # TODO: add your code here
    )
    return main
end

### Testing the subproblem formulation

You can use the cell below to check whether your implementation of the subproblem is correct. For a fixed solution from the main problem, strong duality holds and thus these objective function values should match. We use `≈` which is equivalent to `approx()` to test whether the values are sufficiently close.

In [ ]:
## Test that the primal and dual solutions are the same
(u,v,optval) = generate_and_solve_dual_subproblem(ins, x_bar)
obj = generate_and_solve_primal_subproblem(ins, x_bar)
@test optval ≈ obj

## Benders decomposition algorithm

Here you will combine the functions you defined before into the complete algorithm. 

Some hints:
- You should add a cut before solving the main problem for the first time to make the problem bounded (in the initialisation of the algorithm).
- For the single cut problem, you can ignore the indices $k$ on the lecture slides, as there is only one subproblem being solved.

In [ ]:
function benders_decomposition(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one Benders cut to make the problem bounded
    
    while k <= max_iter && gap > ϵ
        # TODO: obtain necessary solutions
        
        LB = # TODO: what is the lower bound for the objective?
        UB = # TODO: what about the upper bound?
        gap = abs((UB - LB) / UB)
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap)")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

In [ ]:
benders_decomposition(ins)

In [ ]:
objective_value(fullmodel)

# Task 3: 

## Benders components (multi-cut version)

Your task is to create a version of the main problem with multiple Benders cuts being generated at each iteration, and the respective Benders cut. We refer to this version as the multi-cut version. 

Here is a bonus question that might give you ideas on how the implementation could be made more efficient: notice that the previous implementation of the dual subproblem is generating all the cut information at once, and that is why we can reutilise the function `solve_dual_subproblem(ins, x_bar)` here. Imagining that you have a number of parallel computing nodes available, can you see a way that the function `solve_dual_subproblem(ins, x_bar)` could be made more efficient? (bear in mind you are **not required** to implement or try anything in the direction of the answer to the bonus question, but only to give the question a thought!)

In [ ]:
## Benders decomposition: multi-cut

## Generates the main problem
function generate_main_multi(ins)
    
    J = ins.J
    S = ins.S
    V = ins.V
    C = ins.C
    
    # TODO: add your code here
    return main  
end

# Solve the main problem
function solve_main_multi(ins, main)
    
    optimize!(main)
    
    return value.(main[:x]), value.(main[:θ]), objective_value(main)    

end

# Add the Benders cut, given current dual values
function add_benders_cut_multi(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S

    x = main[:x]
    θ = main[:θ]
    
    @constraint(main, [s in S], 
    # TODO: add your code here
    )

    return main
end

In [ ]:
function benders_decomposition_multi(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one set of Benders cuts to make the problem bounded
    
    while k <= max_iter && gap > ϵ
        # TODO: obtain necessary solutions
        
        LB = # TODO: what is the lower bound for the objective?
        UB = # TODO: what about the upper bound?
        
        gap = abs((UB - LB) / UB)
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap)")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

In [ ]:
benders_decomposition_multi(ins)